In [ ]:
# default_exp models.layers

# Layers

> Helper function used to build PyTorch timeseries models.

In [ ]:
#export
from fastai.torch_core import Module
from tsai.imports import *

In [ ]:
#export
def noop(x): return x

In [ ]:
#export
# Misra, D. (2019). Mish: A Self Regularized Non-Monotonic Neural Activation Function. arXiv preprint arXiv:1908.08681.
# https://arxiv.org/abs/1908.08681
# GitHub: https://github.com/digantamisra98/Mish
@torch.jit.script
def mish(input):
    '''Applies the mish function element-wise: mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + exp(x)))'''
    return input * torch.tanh(F.softplus(input))

class Mish(Module):
    def forward(self, input):
        return mish(input)

In [ ]:
class Swish(Module):
    def __init__(self): self.sigmoid = torch.sigmoid
    def forward(self, x): return x.mul_(self.sigmoid(x))

In [ ]:
#export
def get_act_layer(act_fn, act_kwargs={}):
    act_fn = act_fn.lower()
    assert act_fn in ['relu', 'leakyrelu', 'prelu', 'elu', 'mish', 'swish'], 'incorrect act_fn'
    if act_fn == 'relu': return nn.ReLU()
    elif act_fn == 'leakyrelu': return nn.LeakyReLU(**act_kwargs)
    elif act_fn == 'prelu': return nn.PReLU(**act_kwargs)
    elif act_fn == 'elu': return nn.ELU(**act_kwargs)
    elif act_fn == 'mish': return Mish()
    elif act_fn == 'swish': return Swish()

In [ ]:
t = torch.rand(2, 3, 4)
for act_fn in ['relu', 'leakyrelu', 'prelu', 'elu', 'mish', 'swish']: 
    test_eq(get_act_layer(act_fn)(t).shape, t.shape)

In [ ]:
#export
def same_padding1d(seq_len, ks, stride=1, dilation=1):
    effective_ks = (ks - 1) * dilation + 1
    out_dim = (seq_len + stride - 1) // stride
    p = max(0, (out_dim - 1) * stride + effective_ks - seq_len)
    padding_before = p // 2
    padding_after = p - padding_before
    return padding_before, padding_after


class Pad1d(nn.ConstantPad1d):
    def __init__(self, padding, value=0.):
        super().__init__(padding, value)
        

class Conv1dSame(Module):
    "Conv1d with padding='same'"

    def __init__(self, c_in, c_out, ks=3, stride=1, dilation=1, **kwargs):
        self.ks, self.stride, self.dilation = ks, stride, dilation
        self.conv1d_same = nn.Conv1d(c_in, c_out, ks, stride=stride, dilation=dilation, **kwargs)
        self.pad = Pad1d
        

    def forward(self, x):
        self.padding = same_padding1d(x.shape[-1],self.ks,stride=self.stride,dilation=self.dilation)
        return self.conv1d_same(self.pad(self.padding)(x))

In [ ]:
bs = 2
c_in = 3
c_out = 5
seq_len = 6
t = torch.rand(bs, c_in, seq_len)
test_eq(Conv1dSame(c_in, c_out, ks=3, stride=1, dilation=1, bias=False)(t).shape, (bs, c_out, seq_len))

In [ ]:
#export
# https://github.com/locuslab/TCN/blob/master/TCN/tcn.py
class Chomp1d(Module):
    def __init__(self, chomp_size):
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class Conv1dCausal(Module):
    def __init__(self, c_in, c_out, ks, stride=1, dilation=1, **kwargs):
        padding = (ks - 1) * dilation
        self.conv = nn.Conv1d(c_in, c_out, ks, stride=stride, padding=padding, dilation=dilation, **kwargs)
        self.chomp = Chomp1d(math.ceil(padding / stride))

    def forward(self, x):
        return self.chomp(self.conv(x))

In [ ]:
bs = 2
c_in = 3
c_out = 5
seq_len = 512
t = torch.rand(bs, c_in, seq_len)
stride = 1
dilation = 1
test_eq(Conv1dCausal(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape, Conv1dSame(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape)
stride = 1
dilation = 2
test_eq(Conv1dCausal(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape, Conv1dSame(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape)
stride = 2
dilation = 1
test_eq(Conv1dCausal(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape, Conv1dSame(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape)
stride = 2
dilation = 4
test_eq(Conv1dCausal(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape, Conv1dSame(c_in, c_out, ks=3, stride=stride, dilation=dilation)(t).shape)

In [ ]:
#export
def Conv1d(c_in, c_out, ks=3, stride=1, padding='same', dilation=1, bias=True, act_fn='relu', act_kwargs={},
           bn_before_conv=False, bn_before_act=True, bn_after_act=False, zero_bn=False, **kwargs):
    '''conv1d with default padding='same', bn and act_fn (default = 'relu')'''
    layers = []
    if bn_before_conv: layers.append(nn.BatchNorm1d(c_in))
    if padding == 'same': layers.append(Conv1dSame(c_in, c_out, ks, stride=stride, dilation=dilation, bias=bias, **kwargs))
    elif padding == 'causal': layers.append(Conv1dCausal(c_in, c_out, ks, stride=stride, dilation=dilation, bias=bias, **kwargs))
    else:  
        if padding == 'valid': padding = 0
        layers.append(nn.Conv1d(c_in, c_out, ks, stride=stride, padding=padding, dilation=dilation, bias=bias, **kwargs))
    if bn_before_act: layers.append(nn.BatchNorm1d(c_out))
    if act_fn: layers.append(get_act_layer(act_fn, act_kwargs))
    if bn_after_act: 
        bn = nn.BatchNorm1d(c_out)
        nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
        layers.append(bn)
    return nn.Sequential(*layers)

In [ ]:
bs = 2
c_in = 3
c_out = 5
seq_len = 6
ks = 3
t = torch.rand(bs, c_in, seq_len)
test_eq(Conv1d(c_in, c_out, ks=ks, padding=0)(t).shape, (bs, c_out, seq_len - (2 * (ks//2))))
test_eq(Conv1d(c_in, c_out, ks=ks, padding='valid')(t).shape, (bs, c_out, seq_len - (2 * (ks//2))))
test_eq(Conv1d(c_in, c_out, ks=ks, padding='same')(t).shape, (bs, c_out, seq_len))
test_eq(Conv1d(c_in, c_out, ks=ks, padding='causal')(t).shape, (bs, c_out, seq_len))

In [ ]:
Conv1d(c_in, c_out, ks=ks, padding='same')

Sequential(
  (0): Conv1dSame(
    (conv1d_same): Conv1d(3, 5, kernel_size=(3,), stride=(1,))
  )
  (1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)

In [ ]:
Conv1d(c_in, c_out, ks=ks, padding='causal')

Sequential(
  (0): Conv1dCausal(
    (conv): Conv1d(3, 5, kernel_size=(3,), stride=(1,), padding=(2,))
    (chomp): Chomp1d()
  )
  (1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)

In [ ]:
#export
class CoordConv1D(Module):
    def forward(self, x):
        bs, _, seq_len = x.size()
        cc = torch.arange(seq_len, device=device, dtype=torch.float) / (seq_len - 1)
        cc = cc * 2 - 1
        cc = cc.repeat(bs, 1, 1)
        x = torch.cat([x, cc], dim=1)
        return x

In [ ]:
bs = 2
c_in = 3
c_out = 5
seq_len = 6
t = torch.rand(bs, c_in, seq_len)
test_eq(CoordConv1D()(t).shape, (bs, c_in + 1, seq_len))

In [ ]:
#export
class LambdaPlus(Module):
    def __init__(self, func, *args, **kwargs): self.func,self.args,self.kwargs=func,args,kwargs
    def forward(self, x): return self.func(x, *self.args, **self.kwargs)

In [ ]:
#export
class Flatten(Module):
    def forward(self, x): return x.view(x.size(0), -1)
    
class Squeeze(Module):
    def __init__(self, dim=-1):
        self.dim = dim
    def forward(self, x): return x.squeeze(dim=self.dim)
    
class Unsqueeze(Module):
    def __init__(self, dim=-1):
        self.dim = dim
    def forward(self, x): return x.unsqueeze(dim=self.dim)
    
class YRange(Module):
    def __init__(self, y_range:tuple): 
        self.y_range = y_range
        self.sigmoid = torch.sigmoid
    def forward(self, x):
        x = self.sigmoid(x)
        return x * (self.y_range[1] - self.y_range[0]) + self.y_range[0]
    
class Temp(Module):
    def __init__(self, temp):
        self.temp = float(temp)
        self.temp = nn.Parameter(torch.Tensor(1).fill_(self.temp).to(device))
    def forward(self, x):
        return x.div_(self.temp)

In [ ]:
#hide
out = create_scripts()
beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.transforms.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.validation.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_rocket_functions.ipynb.
Converted 100_layers.ipynb.
Converted 100b_models_utils.ipynb.
Converted 101_ResNet.ipynb.
Converted 102_InceptionTime.ipynb.
Converted 103_FCN.ipynb.
Converted 104_ResCNN.ipynb.
Converted index.ipynb.


Checking folder: /Users/nacho/Documents/Machine_Learning/Jupyter_Notebooks/timeseries/tsai
Correct conversion! 😃
Total elapsed time 14 s
Wed, 29 Apr 2020 20:14:37 CEST
 

